In [ ]:
#pip install vaderSentiment

In [46]:
#pip install pattern


In [88]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
import spacy
import re
import nltk
from nltk.tokenize import word_tokenize
nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words -= {"not", "do"}
from autocorrect import Speller
nltk.download('words')
import emoji
import sqlite3
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.corpus import words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import contractions
nltk.download('wordnet')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from pprint import pprint
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Matthew\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matthew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import dataset

In [2]:
conn = sqlite3.connect("IMDB_Movies_2021.db")

cursor = conn.cursor()

table_name = "REVIEWS"

df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

conn.close()

# Drop duplicates and remove nulls

In [3]:
df = df.drop_duplicates(subset = ['REVIEW','AUTHOR'])
df = df.dropna()

# Combine review and title

In [4]:
df['FIXED_REVIEW'] = df['REVIEW'] +df['TITLE']

# Simple preprocesing: lowercase, punctuation, emojis

In [5]:
df['FIXED_REVIEW'] = [review.lower() for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'[^a-zA-Z\s]', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'@\w+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'#\w+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'http\S+|www\S+|https\S+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [emoji.demojize(review) for review in df['FIXED_REVIEW']]

# Remove stop short and long words, lemmatize

In [6]:
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('no')
def sentiment(word):
    blob = TextBlob(word)
    sentiment = blob.sentiment.polarity
    return np.abs(sentiment)

In [7]:
def remove_stop_words(text):
    text = contractions.fix(text)
    words = text.split()    # Tokenize the text into words
    lemmatizer = WordNetLemmatizer()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    filtered_words = [word for word in filtered_words if len(word) > 2]
    filtered_words = [word for word in filtered_words if len(word) < 8]
    filtered_words = [word for word in filtered_words if sentiment(word) != 0 ]
    return ' '.join(filtered_words)

In [8]:
df['FIXED_REVIEW'] = df['FIXED_REVIEW'].apply(remove_stop_words)

# Remove misspelt words (maybe replace with better method)

In [9]:
setofwords = set(words.words())

print("hello" in setofwords)

j = 0
new_review = []

for i in df['FIXED_REVIEW']:
    words = word_tokenize(i)
    new_words = []
    for word in words:
        if word in setofwords:
            new_words.append(word)
    sentence_correct = ' '.join(new_words)        
    new_review.append(sentence_correct)
    #df['FIXED_REVIEW'][j] = sentence_correct
    j+=1


df['FIXED_REVIEW'] = new_review

True


# Add our target (ie: class)

In [10]:
def label_race(row):
    if row['RATING'] >= 7:
        return 2
    if row['RATING'] <= 3:
        return 0
    else:
        return 1

df['CLASS'] = df.apply(label_race, axis=1) 

# Add sentiment as column (can add more)

In [11]:
def get_sentiment_1(text):
    return TextBlob(text).sentiment.polarity 
df['sentiment_1'] = df['FIXED_REVIEW'].apply(get_sentiment_1)

In [34]:
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    return compound_score

def vader_class(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    if compound_score > 0.1:
        sentiment = 2
    elif compound_score < -0.1:
        sentiment = 0
    else:
        sentiment = 1
    
    return sentiment

In [35]:
df['vader_sentiment'] = df['FIXED_REVIEW'].apply(vader_sentiment)
df['vader_class'] = df['FIXED_REVIEW'].apply(vader_class)

# Add a tokenized column for topics

In [14]:
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

In [15]:
df['tokenized_review'] = df['FIXED_REVIEW'].apply(tokenize)

# Now use LDA for topics, gives additional context (may need to optimise how many topics

In [16]:
dictionary = corpora.Dictionary(df['tokenized_review'])
corpus = [dictionary.doc2bow(text) for text in df['tokenized_review']]

In [17]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=20, alpha="auto")

In [18]:
topic_distributions = [lda_model.get_document_topics(doc) for doc in corpus]

In [19]:
topic_matrix = []
for dist in topic_distributions:
    topic_vector = [0] * 10  # Number of topics (in this case 2)
    for topic_id, prob in dist:
        topic_vector[topic_id] = prob
    topic_matrix.append(topic_vector)

# Convert topic_matrix to a DataFrame for easy handling
topic_df = pd.DataFrame(topic_matrix, columns=[f"topic_{i}" for i in range(10)])

In [20]:
df = pd.concat([df, topic_df], axis=1)
df = df.dropna()

# Now BOW with tf idf

In [21]:
v = TfidfVectorizer(ngram_range = (1,2))
bow = v.fit_transform(df.FIXED_REVIEW.values)

In [22]:
bow = pd.DataFrame.sparse.from_spmatrix(bow, columns = v.get_feature_names_out())

In [23]:
df = pd.concat([df, bow], axis=1)


In [24]:
df = df.drop(columns = ['ID','RATING','REVIEW','TITLE'])

In [25]:
df = df.dropna()

# Save data

In [27]:
#df.to_csv('preprocessed_data.csv')

# Using the data in models

In [77]:
#Define X and y
X = df.drop(columns = ['CLASS'])
y = df['CLASS']
X_no_text = df.drop(columns =['AUTHOR','FIXED_REVIEW','tokenized_review'])
X_train = X_no_text

## If method complains about text, then use X_no_text

In [78]:
#Creates correlation matrix of all the columns with the class
def correlation(bow):
    new_corr_df = bow
    new_corr_df['target'] = pd.Series(df['CLASS'])
    target_series = pd.Series(df['CLASS']) ## Maybe 1,5,10 in class
    corr_matrix = new_corr_df.corrwith(target_series)
    return corr_matrix

#Given our correlation matrix, return top k correlated features
def best_k(corr_matrix,k,bow):
    new_corr_df = bow
    top_k = corr_matrix.abs().sort_values(ascending=False)[2:k+2].index.tolist()
    selected_features = new_corr_df[top_k]
    return(selected_features)

# Following code finds top 10 correlated features

In [79]:
X_train = best_k(correlation(X_no_text),10,df) 
X_train

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

...

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Matthew\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: Runt

,vader_sentiment,sentiment_1,vader_class,topic_6,bad,topic_4,topic_0,waste,funny,topic_9
0,-0.2567,-0.099575,0,0.276595,0.234882,0.015275,0.019282,0,0,0.016467
1,0.1494,-0.022222,2,0.272359,0.240108,0.029417,0.496059,0,0,0.031624
2,0.9839,0.329195,2,0.000000,0,0.188105,0.000000,0,0,0.349120
3,0.8856,-0.020238,2,0.013781,0,0.000000,0.410762,0,0,0.169678
4,0.0000,0.116667,1,0.034058,0,0.023819,0.029668,0,0,0.395528
...,...,...,...,...,...,...,...,...,...,...
5181,-0.7436,-0.200000,0,0.000000,0,0.000000,0.000000,0,0,0.000000
5182,-0.2960,-0.150000,0,0.104280,0.092147,0.011213,0.014094,0,0,0.012117
5183,-0.6486,-0.555556,0,0.011311,0,0.000000,0.100076,0,0,0.000000
5184,-0.6249,-0.162500,0,0.117003,0.116228,0.000000,0.000000,0,0,0.068297


# We will need to iterate with different k values to find best k

# Another feature selection method is pca (this finds ten features via pca)

## Again need to find optimum k

In [69]:
k = 10
PCA_TR = PCA(n_components=k)
X_train = pd.DataFrame(PCA_TR.fit_transform(X_no_text))
X_train

C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


# LASSO TBC

# Now onto our models

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2, random_state=27)

## Naive Bayes (multinomial)

In [81]:
model = GaussianNB()
model.fit(X_train, y_train)
model.score(X_test,y_test)

C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.5718503937007874

# Random Forest

In [83]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Accuracy: 0.6033464566929134


C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


## This code will find the best parameters to put into the random forest

In [ ]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier(random_state=20)

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

# Multinomial Logisitc Regression

In [87]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.61      0.76      0.68       392
         1.0       0.00      0.00      0.00       209
         2.0       0.62      0.78      0.69       415

    accuracy                           0.61      1016
   macro avg       0.41      0.51      0.46      1016
weighted avg       0.49      0.61      0.54      1016



C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{met

# Support Vector Machine

In [93]:
clf = svm.SVC(kernel='linear') # Can change the kernel type dependent on performance (rbf, poly)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.60      0.73      0.66       392
         1.0       0.00      0.00      0.00       209
         2.0       0.59      0.77      0.67       415

    accuracy                           0.59      1016
   macro avg       0.40      0.50      0.44      1016
weighted avg       0.47      0.59      0.53      1016



C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Matthew\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in l